In [199]:
#Dataset
input_texts = [
    "How are you?",
    "What's your name?",
    "What do you do?",
    "Ali?"
]

output_texts = [
     "I'm fine, thank you.",
    "My name is John.",
    "i'm engineer",
    "reza"
]

In [200]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

class Translator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Translator, self).__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.encoder = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.decoder = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=2)

    def forward(self, input_seq):

        embedded = self.embedding(input_seq)
        encoder_output, (encoder_hidden, encoder_cell) = self.encoder(embedded)
        
        decoder_output, _ = self.decoder(encoder_output, (encoder_hidden, encoder_cell))

        output = self.fc(decoder_output)
        output = self.softmax(output)

        return output


class TranslationDataset(Dataset):
    def __init__(self, input_texts, output_texts):
        self.input_texts = input_texts
        self.output_texts = output_texts

    def __len__(self):
        return len(self.input_texts)

    def __getitem__(self, index):
        input_text = self.input_texts[index]
        output_text = self.output_texts[index]
        input_seq = text_to_tensor(input_text).squeeze(0)
        output_seq = text_to_tensor(output_text).squeeze(0)

        # pad input_seq and output_seq
        max_len = max(input_seq.shape[0], output_seq.shape[0])
        input_seq = pad_sequences([input_seq], max_len=max_len).squeeze(0)
        output_seq = pad_sequences([output_seq], max_len=max_len).squeeze(0)

        return input_seq, output_seq



def tensor_to_text(tensor):
    text = ''.join([chr(c.item()-1) if 0 <= c.item()-1 <= 0x10FFFF else '' for c in tensor])
    return text


def text_to_tensor(text):
    seq = [ord(c)+1 for c in text]
    tensor = torch.tensor(seq)
    return tensor.unsqueeze(0)

def pad_sequences(sequences, max_len=None, padding_value=0):
    if max_len is None:
        max_len = max(len(seq) for seq in sequences)
    padded_sequences = torch.full((len(sequences), max_len), padding_value)
    for i, seq in enumerate(sequences):
        padded_sequences[i, :len(seq)] = torch.tensor(seq)
    return padded_sequences

def collate_fn(batch):
    # Extracting inputs and outputs from batch
    input_seqs = [item[0] for item in batch]
    target_seqs = [item[1] for item in batch]

    # Find the length of the longest sequence in batch
    max_len = max(max(len(seq) for seq in input_seqs), max(len(seq) for seq in target_seqs))

    # Find sequences with longest sequence length
    input_seqs = pad_sequences(input_seqs, max_len=max_len)
    target_seqs = pad_sequences(target_seqs, max_len=max_len)

    return input_seqs, target_seqs


input_size = 128
hidden_size = 256
output_size = 128
learning_rate = 0.001
num_epochs = 150


translator = Translator(input_size, hidden_size, output_size)
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(translator.parameters(), lr=learning_rate)

dataset = TranslationDataset(input_texts, output_texts)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True,drop_last=True,collate_fn=collate_fn)

for epoch in range(num_epochs):
    for input_seq, target_seq in dataloader:
        optimizer.zero_grad()

        output = translator(input_seq)
        # print(output.view(-1, output_size).shape)
        # print(target_seq.view(-1).shape)
        loss = criterion(output.view(-1, output_size), target_seq.view(-1))

        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")

<ipython-input-200-f73f8c367c91>:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  padded_sequences[i, :len(seq)] = torch.tensor(seq)


Epoch [10/150], Loss: 2.1001980304718018
Epoch [20/150], Loss: 0.062242113053798676
Epoch [30/150], Loss: 0.22997207939624786
Epoch [40/150], Loss: 0.3844526410102844
Epoch [50/150], Loss: 0.3091491758823395
Epoch [60/150], Loss: 0.22025592625141144
Epoch [70/150], Loss: 0.004945266526192427
Epoch [80/150], Loss: 0.01911712996661663
Epoch [90/150], Loss: 0.015034961514174938
Epoch [100/150], Loss: 0.04524823650717735
Epoch [110/150], Loss: 0.03369804471731186
Epoch [120/150], Loss: 0.007843194529414177
Epoch [130/150], Loss: 0.02131364867091179
Epoch [140/150], Loss: 0.006317059975117445
Epoch [150/150], Loss: 0.0052369749173521996


In [221]:
test_input_seq = text_to_tensor("Ali!")
translator.eval()
with torch.no_grad():
    output = translator(test_input_seq)
    
    
    k = 1  # تعداد بهترین پیش‌بینی‌ها که می‌خواهید انتخاب کنید
    _, topk_indices = torch.topk(output, k, dim=2)
    predicted_output_seq = topk_indices.squeeze(0)
      
    # predicted_output_seq = torch.argmax(output, dim=2)
    predicted_output_text = tensor_to_text(predicted_output_seq.squeeze())

    print("Input: Ali?")
    print("Predicted Output:", predicted_output_text)

Input: Ali?
Predicted Output: reza
